In [1]:
from dotenv import load_dotenv
import dspy
import mlflow

from agentic_system.agents import (
    CFEfficacyAgent,
    ToxicityScreeningAgent,
    CompoundPrioritizationAgent,
)

In [2]:
# NOTE: Start MLflow server with:
# mlflow server --backend-store-uri sqlite:///mydb.sqlite
# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Create a unique name for your experiment.
mlflow.set_experiment("Optimize Efficacy Tools")
mlflow.autolog()
mlflow.tracing.disable_notebook_display()

load_dotenv("../.env")
lm = dspy.LM("gemini/gemini-2.5-pro", temperature=0.9, cache=False)
dspy.settings.configure(lm=lm, track_usage=True)

2025/09/17 16:12:39 INFO mlflow.tracking.fluent: Autologging successfully enabled for dspy.
2025/09/17 16:12:39 INFO mlflow.tracking.fluent: Autologging successfully enabled for litellm.
2025/09/17 16:12:39 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [3]:
COMPOUND = "SU11274"

efficacy_agent = CFEfficacyAgent()
efficacy_results = efficacy_agent(compound_name=COMPOUND)
print(efficacy_results)

Prediction(
    trajectory={'thought_0': "I need to determine the efficacy of SU11274 in a cardiac fibroblast assay. To do this, I will first identify the compound's mechanism of action and its primary targets. Then, I will investigate the role of these targets in cardiac fibroblast biology and heart failure. This information will allow me to predict the compound's effect on the failing fibroblast phenotype. I will start by finding the ChEMBL ID for SU11274.", 'tool_name_0': 'CHEMBL__search_chembl_id', 'tool_args_0': {'query': 'SU11274', 'goal': 'Find the ChEMBL ID for the compound SU11274.'}, 'observation_0': 'The ChEMBL ID for SU11274 is CHEMBL261641.', 'thought_1': 'Now that I have the ChEMBL ID for SU11274 (CHEMBL261641), I will investigate its mechanism of action to understand its biological targets. This will be the first step in assessing its potential to reverse the failing cardiac fibroblast phenotype.', 'tool_name_1': 'CHEMBL__get_mechanisms_of_action', 'tool_args_1': {'chemb

In [6]:
steps = len(efficacy_results.trajectory) // 4
cost = sum([x["cost"] for x in lm.history])

print("Steps:", steps)
print("Total Cost (USD):", cost)

Steps: 5
Total Cost (USD): 0.08049500000000001


In [6]:
# tox_agent = ToxicityScreeningAgent(max_iters=12)
# tox_results = tox_agent(compound_name=COMPOUND)
# print(tox_results)

In [7]:
# prioritization_agent = CompoundPrioritizationAgent()
# prioritization_results = prioritization_agent(compound_name=COMPOUND)
# print(prioritization_results)